In [9]:
import numpy as np
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, isnull, col, sum, when


os.environ["PYSPARK_PYTHON"] = 'C:\\Users\\Neesty\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\ml-sim-kc-8z4rnjSt-py3.11\\Scripts\\python.exe'
# Создаем SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()

# Пример данных (замените его своим датафреймом)
data = [
    (0, None, 10),
    (1, 20, 30),
    (2, 40, None),
    (3, 50, 12),
]

columns = ["id", "col1", "col2"]

# Создаем PySpark DataFrame
df = spark.createDataFrame(data, columns)
print(df.show())
# Проверяем наличие NaN значений в конкретном столбце (например, 'col1')
# df.select(isnan("col1")).show()
#
# # Проверяем наличие null значений в конкретном столбце (например, 'col2')
# df.select(isnull("col2")).show()

# Проверяем наличие NaN или null значений во всех столбцах
# df.select([isnan(c) | isnull(c) for c in df.columns]).show()
missing_counts = df.select([sum(when(isnan(c) | col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns])

# Вывод результатов
missing_counts.show()

+---+----+----+
| id|col1|col2|
+---+----+----+
|  0|NULL|  10|
|  1|  20|  30|
|  2|  40|NULL|
|  3|  50|  12|
+---+----+----+

None
+---+----+----+
| id|col1|col2|
+---+----+----+
|  0|   1|   1|
+---+----+----+



In [2]:
import sys

# Вывести путь к исполняемому файлу Python
sys.executable

'C:\\Users\\Neesty\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\ml-sim-kc-8z4rnjSt-py3.11\\Scripts\\python.exe'

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

# Создаем Spark сессию
spark = SparkSession.builder.appName("example").getOrCreate()

# Пример DataFrame
data = [("Alice", 1, None), ("Bob", None, None), ("Charlie", None, 5)]
columns = ["name", "column1", "column2"]
df = spark.createDataFrame(data, columns,)
df.show()
# Подсчет строк с пропущенными значениями
exprs = [expr(f"sum(if({c} is null or {c} = 'None', 1, 0)) as {c}_count") for c in df.columns]

# Выполняем selectExpr для подсчета строк с пропущенными значениями в каждой колонке
result = df.selectExpr(*exprs).collect()

# Суммируем результаты по всем колонкам
total_rows_with_null = sum(result[0])
# Вывод результата
print(f"Количество строк с пропущенными значениями: {total_rows_with_null}")


+-------+-------+-------+
|   name|column1|column2|
+-------+-------+-------+
|  Alice|      1|   NULL|
|    Bob|   NULL|   NULL|
|Charlie|   NULL|      5|
+-------+-------+-------+



PySparkTypeError: [NOT_ITERABLE] Column is not iterable.